# Lesson - SQL Indexing

In this lesson, we'll explore how queries are executed in SQLite. After exploring this at a high level, we will explore how to create and use indexes for better performance. As our data gets larger and our queries more complex, it's important to be able to tweak the queries we write and optimize a database's schema to ensure that we're getting results back quickly.

To explore database performance, we'll work with `factbook.db`, a SQLite database that contains information about each country in the world. We'll be working with the `facts` table in the database. Each row in facts represents a single country, and contains several columns, including:

`name` -- the name of the country.
`area` -- the total land and sea area of the country.
`population` -- the population of the country.
`birth_rate` -- the birth rate of the country.
`created_at` -- the date the record was created.
`updated_at` -- the date the record was updated.

**Exercise - Setting up Environment**
We will work with sqlite from python.
- Write a query that returns the schema of the facts table and assign the resulting list of tuples to schema.
- Use a for loop and a print statement to display each tuple in schema on a separate line.

In [10]:
import sqlite3
conn=sqlite3.connect("factbook.db")

schema=conn.execute("pragma table_info(facts);").fetchall() #we can directly run query without instantiating a cursor obect
for s in schema:
    print(s)


(0, 'id', 'INTEGER', 1, None, 1)
(1, 'code', 'varchar(255)', 1, None, 0)
(2, 'name', 'varchar(255)', 1, None, 0)
(3, 'area', 'integer', 0, None, 0)
(4, 'area_land', 'integer', 0, None, 0)
(5, 'area_water', 'integer', 0, None, 0)
(6, 'population', 'integer', 0, None, 0)
(7, 'population_growth', 'float', 0, None, 0)
(8, 'birth_rate', 'float', 0, None, 0)
(9, 'death_rate', 'float', 0, None, 0)
(10, 'migration_rate', 'float', 0, None, 0)


### Query Planner
When we execute a SQL query, SQLite performs many steps before returning the results. First, it tokenizes and parses the query to look for any syntax errors. If there are any syntax errors, the query execution process halts and the error message is returned to you. If the parser was able to successfully parse the query, then SQLite moves on to the query planning and optimization phase.

When working with a database that's stored on disk as a file, it's crucial to minimize the amount of disk reads necessary to avoid long running times. The **query optimizer** generates cost estimates for the various ways to access the underlying data, factoring in the schema of the tables and the operations the query requires.

The **optimizer** quickly assesses the various ways to access the data and generates a best guess for the fastest query plan. This high level query plan is then converted into highly efficient, lower-level C code to interact with the database file on disk. We can observe the query plan to understand what SQLite is doing to return our results.

### `EXPLAIN` QUERY PLAN

We can use the `EXPLAIN QUERY PLAN` statement before any query we're running to get a high level query plan that would be performed. If you write a SELECT statement and place the EXPLAIN QUERY PLAN statement before it:
`EXPLAIN QUERY PLAN SELECT * FROM facts;`
the results of the SELECT query won't be returned and instead the high level query plan will be:
```
[(0, 0, 0, 'SCAN TABLE facts')]

```

We'll focus on `'SCAN TABLE facts'`, the last value from the returned tuple. `SCAN TABLE` means that every row in the entire table (facts) had to be accessed to evaluate the query. Since the SELECT query we wrote returns all of the columns and rows in the facts table, the entire table had to be accessed to get the results we requested.

  When running the query using the `sqlite3` library, we'll need to use the `fetchall()` method.
  ```
  query_plan = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts;").fetchall()
  ```
The query plan is represented as a tuple, which is the sqlite3 library's preferred way of representing results.

**Exercise**
- Return the query plan for the query that returns all columns and rows where area exceeds 40000. Assign the results to `query_plan_one`.

- Return the query plan for the query that returns only the `area` column for all rows where area exceeds 40000. Assign the results to `query_plan_two`.

- Return the query plan for the query that returns the row for the country Czech Republic. Assign the results to `query_plan_three`.

- Use the print function to display each query plan.

In [14]:
import sqlite3
conn=sqlite3.connect("factbook.db")

query_one="EXPLAIN QUERY PLAN SELECT * FROM facts WHERE area>40000;"
query_plan_one=conn.execute(query_one).fetchall()

query_two="EXPLAIN QUERY PLAN SELECT area FROM facts WHERE area>40000;"
query_plan_two=conn.execute(query_two).fetchall()

query_three="EXPLAIN QUERY PLAN SELECT * FROM facts WHERE name='Czech Republic';"
query_plan_three=conn.execute(query_three).fetchall()

print(query_plan_one)
print(query_plan_two)
print(query_plan_three)

[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]


### Data Representation
all 3 query plans are exactly the same. The entire facts table had to be accessed to return the data we needed for all 3 queries. Even though all the queries asked for a subset of the facts table, SQLite still ends up scanning the entire table. This is because of the way SQLite represents data.

For the facts table, we set the `id` column as the `primary key` and SQLite uses this column to order the records in the database file. Since the rows are ordered by `id`, SQLite can search for a specific row based on it's `id` value using binary search. Unless we provide specific `id` values in the `WHERE` statement in the query, SQLite can't take advantage of binary search and has to instead scan the entire table, row by row.

To return the results for the first 2 queries, SQLite has to:

- Access the first row in the table (lowest id value),
- check if that row's value for area exceeds 40000 and store the row separately in a temporary collection if it is,
move onto the next row,
- check if that row's value for area exceeds 40000 and store the row separately in a temporary collection if it is,
- repeat moving and checking each row for the rest of the table,
- return the final collection of rows that meet the criteria.

If we were instead interested in a row with a specific id value, like in the third query:
`SELECT * FROM facts WHERE id=15;`

SQLite can use binary search to quickly find the corresponding row at that id value. Instead of performing a full table scan, SQLite would:

- use binary search to find the first row where the id value matches 15 in `O(log N)` time complexity and store this row in a temporary collection,
- advance to the next row to look for any more rows with the same id values and add those rows to the temporary collection,
- return the final collection of rows that matched.

If we set the `id` column to be a `UNIQUE PRIMARY KEY` when we created the schema, SQLite would stop searching when it found the instances that matched the `id` value. It would avoid advancing to the next row(s) since no 2 rows could have the same id value. While we didn't enforce the `UNIQUE` constraint on the `id` column, all of the values currently in the column are in fact unique and SQLite will only have to advance one row to realize this since they're ordered.

### Time Complexity

Binary search on a table using the primary key would be `O(log N)` time complexity where `N` is the number of total rows in the table. On the other hand, a full table scan would would be `O(N)` time complexity since each row would have to be accessed. If we're working with a database containing millions of rows, binary search would be over a million times faster! 

**Exercise**
Return the query plan for the query that selects the row at `id value 20` from the `facts` table.
Assign the query plan to `query_plan_four` and display the query plan using the print function.

In [1]:
import sqlite3
conn=sqlite3.connect("factbook.db")

query_four="EXPLAIN QUERy PLAN SELECT * FROM facts WHERE id=20;"
query_plan_four=conn.execute(query_four).fetchall()
print(query_plan_four)

[(2, 0, 0, 'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]


### Search and rowid

Instead of using a full table scan:
`[(0, 0, 0, 'SCAN TABLE facts')]`

SQLite performed binary search on the `facts` table using the integer primary key:
`[(0, 0, 0, 'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]`

SQLite uses `rowid` to refer to the primary key of a table. The alias `rowid` will be displayed in the query plan, no matter what you name the primary key column for that table. Either `SCAN` or `SEARCH` will always appear at the start of the query explanation for SELECT queries.

### Indexing

SQLite can take advantage of speedy lookups when searching for a specific primary key. Unfortunately, we don't always have the primary keys for the rows we're interested in beforehand. When we're expressing our intent as a SQL query, we're often thinking in terms of row and column values. We need to find a way that allows us to benefit from the speed of primary key lookups without actually knowing the primary key in advance.

To that end, we could create a separate table that's optimized for lookups by a different column from the facts table instead of by the id. We can make the column we want to query by the primary key, so we get the speed benefits, and embed the id value from the facts table corresponding to that row. We call this table an `index` and each row in the `index` contains:

- the value we want to be able to search by, as the primary key,
- an id value for the corresponding row in facts.

If we wrote a `SELECT` query to look up the population of India from the facts table:
`SELECT population FROM facts WHERE name = 'India';`

SQLite would need to perform a full table scan on `facts` to find the specific row where the value for name was India. We can instead create an `index` that's ordered by name values (primary key) and where each row contains the corresponding row's `id` from the facts table. Here's what that index would look like:




### Creating an Index

Instead of creating a separate table and updating it ourselves, we can specify a column we want an index table for and `SQLite` will take care of the rest. `SQLite`, and most databases, make it easy for you to create indexes for tables on columns we plan to query often. To create an index we use the `CREATE INDEX` statement:
`CREATE INDEX index_name ON table_name(column_name);`

Each index we create needs a name (to replace index_name). Similar to when we add a table to a database, using the `IF NOT EXISTS` clause helps us avoid attempting to create an index that already exists. Doing so will cause SQLite to throw an error. To create an index for the `area` column called `area_idx`, we write the following query:
```
CREATE INDEX IF NOT EXISTS area_idx ON facts(area);
```
An empty array will be returned when we run the query. The main benefit of having `SQLite` handle the maintenance of indexes we create is that the indexes are used automatically when we execute a query whenever there will be any speed advantages. As our queries become more complex, letting SQLite decide how and when to use the indexes we create helps us be much more productive.

If we create an index for the `area` column in the `facts` table, `SQLite` will use the index whenever we search for rows in facts using that column. Each row would only contain the `area` value and the corresponding row's `id` value. The index would be ordered by the area values for quick lookups.

All three of the following queries would take advantage of the area_idx index:
```
SELECT * FROM facts WHERE area = 10000;
SELECT * FROM facts WHERE area > 10000;
SELECT * FROM facts WHERE area < 10000;
```
Since the area_idx index would be ordered by the area values, SQLite would:

- search for the first instance in the index where area equaled 10000 and store the `id` value in a temporary collection.
- it would then advance to the next row in the index to check if the `WHERE` condition was still met.
- if not, then the temporary collection would be returned and the process completes.
- if so, then SQLite would add that `id` value to the collection and check the next row.
- when SQLite finds a value for area that doesn't match the `WHERE` condition,
 - it will look up and return the rows in facts using the id values stored in the temporary collection.
 - each of these lookups will be `O(log N)` time complexity and while this could add up, it will still be faster than a full table scan.
 
This process allows us to just write one query instead of 2 and have `SQLite` maintain and interact with the index. A table can have many indexes, and most tables in production environments usually do have many indexes. **Every time we add or delete a row to the table, all of the indexes will be updated**. If we edit the values in a row, SQLite will figure out which indexes are affected by the changes and update those indexes.

While creating indexes gives us tremendous speed benefits, they come at the cost of space. Each index needs to be stored in the database file. In addition, adding, editing, and deleting rows takes longer since each of the affected indexes need to be updated. Since indexes can be created after a table is created, it's recommended to only create an index while querying on a specific column frequently.

**Exercise**

- Return the query plan for the query that returns all values in the rows in `facts` where the population exceeds 10000. Assign the resulting query plan to `query_plan_six` and display using the print function.
- Create an index for the population column in the facts table named pop_idx.
- Return the query plan for the query that returns all values in the rows in facts where the population exceeds 10000. Assign the resulting query plan to query_plan_seven and display using the print function.

In [2]:
import sqlite3
conn=sqlite3.connect("factbook.db")

query_six="EXPLAIN QUERy PLAN SELECT * FROM facts WHERE population>10000;"
query_plan_six=conn.execute(query_six).fetchall()

conn.execute("CREATE INDEX IF NOT EXISTS pop_idx ON facts(population);")

query_seven="EXPLAIN QUERy PLAN SELECT * FROM facts WHERE population>10000;"
query_plan_seven=conn.execute(query_seven).fetchall()

print(query_plan_six)
print(query_plan_seven)

[(2, 0, 0, 'SCAN TABLE facts')]
[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


Instead of ending in `USING INDEX pop_idx (population)`, the query plan ended in `USING INDEX pop_idx (population>?)`. This is to indicate the granularity of the lookup that SQLite had to do for that index.